# bert-financial-sentiment

----

In [ ]:
# 使用 pip3 工具安装 transformers 库，它提供了各种预训练模型、tokenizer 和其他工具，方便用户进行自然语言处理任务。
!pip install transformers 

# 使用 pip3 工具安装 ipywidgets 库，它提供了一系列交互式小部件，用于在 Jupyter Notebook 中创建动态和交互式的用户界面。
!pip install ipywidgets 

**解释：**

这段代码使用 `pip3` 工具安装了两个 Python 库： `transformers` 和 `ipywidgets`。

* **`transformers`** 库： 用于自然语言处理 (NLP) 的一个强大工具包，提供各种预训练模型、tokenizer 和其他工具，方便用户进行 NLP 任务，例如文本分类、文本生成、机器翻译等。它包含了各种预训练模型，例如 BERT、GPT-2、RoBERTa 等等，可以根据用户的任务需求选择合适的模型进行微调或使用。
* **`ipywidgets`** 库： 用于在 Jupyter Notebook 环境中提供交互式小部件，让用户能够通过鼠标或键盘与代码进行交互。它可以用来创建动态和交互式的用户界面，提高代码的可视化效果，并使代码更易于调试和理解。

这段代码的主要作用是安装 `transformers` 和 `ipywidgets` 两个库，为后续使用这些库进行 NLP 任务和 Jupyter Notebook 交互式操作做好准备。


----

In [ ]:
# 导入 logging 库，用于记录程序运行过程中的信息和错误。
import logging 

# 导入 os 库，用于操作文件系统。
import os 

# 导入 sys 库，用于访问系统特定的参数和函数。
import sys 

# 导入 numpy 库，用于处理数组和矩阵运算。
import numpy as np 

# 导入 pandas 库，用于读取和处理数据表格。
import pandas as pd 

# 导入 PyTorch 库，用于构建和训练深度学习模型。
import torch 

# 导入 PyTorch 的 DataLoader 和 TensorDataset 类，用于构建数据加载器和张量数据集。
from torch.utils.data import DataLoader, TensorDataset 

# 导入 Hugging Face 的 transformers 库的 AdamW、BertForSequenceClassification 和 BertTokenizer 类，
# 用于使用预训练的 BERT 模型进行文本分类任务。
from transformers import AdamW, BertForSequenceClassification, BertTokenizer 

# 导入 scikit-learn 的 OrdinalEncoder 类，用于将分类特征转换为数值型特征。
from sklearn.preprocessing import OrdinalEncoder 

# 导入 scikit-learn 的 train_test_split 函数，用于将数据集分成训练集和测试集。
from sklearn.model_selection import train_test_split 

# 导入 SimpleNamespace 类，用于创建命名空间对象，方便组织代码结构。
from types import SimpleNamespace 

# 创建一个名为 __name__ 的日志记录器。
logger = logging.getLogger(__name__) 

# 设置日志记录器的级别为 DEBUG，即记录所有级别的信息。
logger.setLevel(logging.DEBUG) 

# 将日志信息输出到标准输出流。
logger.addHandler(logging.StreamHandler(sys.stdout)) 

**解释：**

这段代码导入了所需的库，并创建了一个日志记录器，用于在程序运行过程中记录信息和错误。

* **`logging` 库：**  用于在程序中记录信息和错误，方便调试和跟踪程序运行过程。
* **`os` 库：** 用于与操作系统交互，访问文件系统、环境变量等。
* **`sys` 库：**  提供与 Python 解释器相关的信息和函数，例如访问命令行参数、获取系统平台信息等。
* **`numpy` 库：**  用于处理数组和矩阵运算，提供高效的数值计算功能。
* **`pandas` 库：**  用于读取和处理数据表格，提供灵活的数据操作和分析功能。
* **`torch` 库：**  用于构建和训练深度学习模型，提供了张量运算、自动微分、神经网络等功能。
* **`DataLoader` 和 `TensorDataset` 类：** PyTorch 中的用于构建数据加载器和张量数据集，方便模型训练时加载数据。
* **`transformers` 库：** Hugging Face 提供的用于自然语言处理任务的库，包含各种预训练模型、tokenizer 和其他工具，方便用户快速构建和使用 NLP 模型。
* **`AdamW`、`BertForSequenceClassification` 和 `BertTokenizer` 类：**  `transformers` 库中用于使用预训练 BERT 模型进行文本分类任务的工具。
* **`OrdinalEncoder` 类：**  `sklearn` 库中用于将分类特征转换为数值型特征。
* **`train_test_split` 函数：**  `sklearn` 库中用于将数据集分成训练集和测试集。
* **`SimpleNamespace` 类：**  用于创建命名空间对象，方便组织代码结构，将相关的变量和函数归类到同一个对象下。
* **日志记录器：** 创建一个名为 `__name__` 的日志记录器，并设置其日志级别为 `DEBUG`，将日志信息输出到标准输出流。

这段代码的主要作用是导入必要的库，并设置日志记录器，方便开发者调试和跟踪程序运行过程。后续的代码将会使用这些库中的工具进行自然语言处理和模型训练等任务。


----

In [ ]:
# 设置数据集文件路径。
filepath = './data/all-data.csv' 

# 读取数据集文件，并使用 ISO-8859-1 编码进行解码，
# 只读取第一列和第二列数据，并分别命名为 "sentiment" 和 "article"。
data = pd.read_csv(filepath, encoding="ISO-8859-1", 
header=None, usecols=[0, 1], 
names=["sentiment", "article"]) 

# 创建一个 OrdinalEncoder 对象，用于将分类特征转换为数值型特征。
ord_enc = OrdinalEncoder() 

# 使用 OrdinalEncoder 对 "sentiment" 列进行编码，并将其存储在 "sentiment" 列中。
data["sentiment"] = ord_enc.fit_transform(data[["sentiment"]]) 

# 将 "sentiment" 列的数据类型转换为整数类型。
data = data.astype({'sentiment':'int'}) 

# 将数据集分成训练集和测试集，并使用默认比率进行分割。
train, test = train_test_split(data) 

# 将训练集保存到文件 "train.csv"，不保存索引信息。
train.to_csv("./data/train.csv", index=False)

# 将测试集保存到文件 "test.csv"，不保存索引信息。
test.to_csv("./data/test.csv", index=False)

# 计算所有文本的最大长度，并将其存储在 MAX_LEN 变量中。
MAX_LEN = data.article.str.len().max()

**解释：**

这段代码完成以下几项工作：

1. **读取数据：** 从 `all-data.csv` 文件中读取数据，将第一列作为情感标签（`sentiment`），第二列作为文本内容（`article`）。
2. **预处理数据：**  使用 `OrdinalEncoder` 将情感标签转换成数值型特征，将情感类别转换为整数类型，并使用 `train_test_split` 函数将数据分成训练集和测试集。
3. **保存数据：**  将训练集和测试集分别保存到`train.csv`和`test.csv`文件中。
4. **计算文本最大长度：** 计算所有文本内容的最大长度，并将其存储在 `MAX_LEN` 变量中，用于后续模型训练时的文本截断和填充。

**代码逻辑：**

这段代码使用`pandas`库读取和处理`all-data.csv`文件，并使用`scikit-learn`库对数据进行预处理，最后将数据分割成训练集和测试集，并存储到相应的 CSV 文件中。

* `pd.read_csv(filepath, encoding="ISO-8859-1", header=None, usecols=[0, 1], names=["sentiment", "article"])`：读取 CSV 文件，设置编码方式、不使用标题行、只读取第一列和第二列，并分别命名为 "sentiment" 和 "article"。
* `ord_enc.fit_transform(data[["sentiment"]])`：使用 OrdinalEncoder 将 "sentiment" 列进行编码，并将结果存储在 "sentiment" 列中。
* `data.astype({'sentiment':'int'})`：将 "sentiment" 列的数据类型转换成整数类型。
* `train, test = train_test_split(data)`：将数据集分成训练集和测试集。
* `train.to_csv("./data/train.csv", index=False)`：将训练集保存到文件 "train.csv"，不保存索引信息。
* `test.to_csv("./data/test.csv", index=False)`：将测试集保存到文件 "test.csv"，不保存索引信息。
* `data.article.str.len().max()`： 计算所有文本内容的最大长度，并将其存储在 MAX_LEN 变量中。


这段代码为后续的模型训练和测试做准备工作，将原始数据整理成可以被模型使用的格式，并存储到相应的文件中。


----

In [ ]:
def get_data_loader(batch_size, training_dir, filename): 
    # 记录日志信息。
    logger.info("Get data loader") 

    # 加载预训练的 BERT tokenizer。
    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True) 

    # 从指定的目录和文件名加载数据。
    dataset = pd.read_csv(os.path.join(training_dir, filename)) 

    # 获取所有文本内容和情感标签。
    articles = dataset.article.values 
    sentiments = dataset.sentiment.values 

    # 初始化一个列表，用于存储所有文本的 token id。
    input_ids = [] 

    # 遍历所有文本，将其转换成 token id 并添加到列表中。
    for sent in articles: 
        encoded_articles = tokenizer.encode(sent, add_special_tokens=True) 
        input_ids.append(encoded_articles) 

    # 对较短的文本进行填充，使其长度与最长的文本长度一致。
    input_ids_padded = [] 
    for i in input_ids: 
        while len(i) < MAX_LEN: 
            i.append(0) 
        input_ids_padded.append(i) 
    input_ids = input_ids_padded 

    # 创建一个列表，用于存储所有文本的注意力掩码。
    # 注意力掩码用于标识文本中的有效 token，在 BERT 模型中用于控制注意力机制。
    attention_masks = [] 

    # 遍历所有 token id，生成相应的注意力掩码。
    # 如果 token id 大于 0，则表示该 token 是有效的，对应掩码值为 1，反之则为 0。
    for sent in input_ids: 
        att_mask = [int(token_id > 0) for token_id in sent] 
        attention_masks.append(att_mask) 

    # 将 token id、注意力掩码和情感标签转换为 PyTorch 张量。
    train_inputs = torch.tensor(input_ids) 
    train_labels = torch.tensor(sentiments) 
    train_masks = torch.tensor(attention_masks) 

    # 使用 PyTorch 的 TensorDataset 类创建数据集，并使用 DataLoader 类创建数据加载器。
    tensor_data = TensorDataset(train_inputs, train_masks, train_labels) 
    tensor_dataloader = DataLoader(tensor_data, batch_size=batch_size) 

    # 返回数据加载器。
    return tensor_dataloader 

**解释：**

这段代码定义了一个名为 `get_data_loader` 的函数，该函数用于从指定路径加载数据，并将其转换为 PyTorch 数据加载器，以便用于模型训练。

* **函数参数：**  该函数接受三个参数：`batch_size` 用于指定每个 batch 中包含的样本数量，`training_dir` 用于指定存储数据的目录，`filename` 用于指定数据文件名。
* **加载数据：**  函数首先从指定的目录和文件名中加载 CSV 数据，并获取所有文本内容和情感标签。
* **预处理数据：**
    * 首先使用 BERT tokenizer 将每个文本转换成 token id。
    * 然后对较短的文本进行填充，使其长度与最长的文本长度一致，以便使用 BERT 模型进行处理。
    * 最后生成注意力掩码，用于标识文本中的有效 token，以便 BERT 模型能够专注于处理有效的部分。
* **创建数据集和数据加载器：**  将 token id、注意力掩码和情感标签转换成 PyTorch 张量，并使用 `TensorDataset` 类创建数据集。最后使用 `DataLoader` 类创建数据加载器，以便将数据以 batch 的形式加载到模型中，用于训练。

这段代码示例了如何使用 Hugging Face 提供的 BERT 工具包和 PyTorch 库对文本数据进行预处理和数据加载，以便将数据有效地使用在模型训练中。


----

In [ ]:
def train(args): 
    # 判断是否使用 GPU 进行训练。
    use_cuda = args.num_gpus > 0 

    # 设置训练设备，如果使用 GPU 则使用 CUDA，否则使用 CPU。
    device = torch.device("cuda" if use_cuda else "cpu") 

    # 设置随机数种子，以确保每次训练结果一致。
    torch.manual_seed(args.seed) 
    if use_cuda: 
        torch.cuda.manual_seed(args.seed) 

    # 加载训练数据集和测试数据集。
    train_loader = get_data_loader(args.batch_size, args.data_dir, args.train_file) 
    test_loader = get_data_loader(args.test_batch_size, args.data_dir, args.test_file) 

    # 加载预训练的 BERT 模型，并设置模型参数。
    model = BertForSequenceClassification.from_pretrained( 
        "bert-base-uncased",   
        num_labels=args.num_labels,  
        output_attentions=False,   
        output_hidden_states=False,  ) 

    # 将模型加载到指定设备上。
    model = model.to(device) # load the model to the right device 

    # 配置优化器。
    optimizer = AdamW( 
        model.parameters(), 
        lr=args.lr,  # learning rate  
    ) 

    # 开始训练循环。
    for epoch in range(1, args.epochs + 1): 
        # 初始化本轮训练的累计损失为 0。
        total_loss = 0 

        # 将模型设置为训练模式。
        model.train() 

        # 遍历训练数据加载器中的每个 batch 的数据。
        for step, batch in enumerate(train_loader): 
            # 将 batch 数据加载到指定设备上。
            b_input_ids = batch[0].to(device) 
            b_input_mask = batch[1].to(device) 
            b_labels = batch[2].to(device) 

            # 清除模型参数的梯度信息。
            model.zero_grad() 

            # 将数据输入模型进行训练。
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels) 

            # 获取损失值。
            loss = outputs[0] 

            # 将当前 batch 的损失值累加到 total_loss 中。
            total_loss += loss.item() 

            # 反向传播计算损失函数对模型参数的梯度。
            loss.backward() 

            # 进行梯度裁剪，防止梯度爆炸。
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0) 

            # 根据梯度信息更新模型参数，并进行一步优化。
            optimizer.step() 

            # 每隔一定步数打印训练信息。
            if step % args.log_interval == 0: 
                logger.info( 
                    "训练周期: {} [{}/{} ({:.0f}%)] 损失: {:.6f}".format( 
                        epoch, 
                        step * len(batch[0]), 
                        len(train_loader.sampler), 
                        100.0 * step / len(train_loader), 
                        loss.item(), 
                    ) 
                ) 

        # 打印本轮训练的平均损失。
        logger.info("平均训练损失: %f\n", total_loss / len(train_loader))  

        # 在测试集上评估模型。
        test(model, test_loader, device) 

        # 保存训练好的模型。
        logger.info("保存训练好的模型.") 
        model_2_save = model.module if hasattr(model, "module") else model 
        model_2_save.save_pretrained(save_directory=args.model_dir) 

        # 返回训练好的模型。
        return model 

**解释：**

这段代码定义了一个名为 `train` 的函数，该函数用于训练一个 BERT 模型，并使用训练好的模型在测试集上进行评估，最后将训练好的模型保存到指定目录。

* **函数参数：** 函数接收一些参数，包括训练轮数、学习率、批次大小、数据路径、日志间隔等等，这些参数用于控制训练过程。
* **初始化：** 函数首先根据参数设置训练设备（CPU 或 GPU）、随机数种子和训练数据集和测试数据集。
* **加载模型：** 函数加载预训练的 BERT 模型，并设置模型参数，包括标签数量等。
* **优化器：** 函数使用 AdamW 优化器进行模型参数的更新。
* **训练循环：** 函数使用循环遍历训练数据集，并进行以下操作：
    * 清除梯度信息
    * 将数据输入模型进行训练
    * 计算损失值
    * 反向传播计算梯度
    * 梯度裁剪（防止梯度爆炸）
    * 更新模型参数
    * 打印训练信息
* **评估模型：** 函数在每个 epoch 结束后，使用训练好的模型在测试集上进行评估，并打印评估结果。
* **保存模型：** 函数最后将训练好的模型保存到指定的目录中。

这段代码展示了如何使用 BERT 模型进行文本分类任务的训练，以及如何使用 PyTorch 进行模型训练和评估。


----

In [ ]:
def test(model, test_loader, device):     

    # 定义一个函数，用于计算预测结果与真实标签之间的准确率。
    def get_correct_count(preds, labels): 
        # 将预测结果和真实标签转换成一维数组。
        pred_flat = np.argmax(preds, axis=1).flatten() 
        labels_flat = labels.flatten() 
        # 计算预测正确的数量。
        return np.sum(pred_flat == labels_flat), len(labels_flat) 

    # 将模型设置为评估模式。
    model.eval() 

    # 初始化准确率和样本数量为 0。
    _, eval_accuracy = 0, 0 
    total_correct = 0 
    total_count = 0 

    # 使用 torch.no_grad() 上下文管理器来禁用梯度计算，因为在评估过程中不需要计算梯度。
    with torch.no_grad(): 
        # 遍历测试数据集的每个 batch。
        for batch in test_loader: 
            # 将 batch 数据加载到指定设备上。
            b_input_ids = batch[0].to(device) 
            b_input_mask = batch[1].to(device) 
            b_labels = batch[2].to(device) 

            # 使用模型进行预测。
            outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask) 
            preds = outputs[0] 
            # 将预测结果转换为 numpy 数组。
            preds = preds.detach().cpu().numpy() 
            label_ids = b_labels.to("cpu").numpy() 

            # 使用 get_correct_count 函数计算当前 batch 的准确率。
            num_correct, num_count = get_correct_count(preds, label_ids) 
            # 累加预测正确的数量和样本数量。
            total_correct += num_correct 
            total_count += num_count 

    # 打印测试集上的准确率。
    logger.info("测试集: 准确率: %f\n", total_correct/total_count) 

**解释：**

这段代码定义了一个名为 `test` 的函数，用于测试训练好的 BERT 模型，并计算其在测试集上的准确率。

* **函数参数：**  函数接收三个参数：`model` 为训练好的模型，`test_loader` 为测试数据集的加载器，`device` 用于指定使用的设备（CPU 或 GPU）。
* **计算准确率函数：**  定义了一个名为 `get_correct_count` 的子函数，用于计算预测结果和真实标签之间的准确率。
* **模型评估：**
    * 函数首先将模型设置为评估模式，禁用梯度计算。
    * 然后遍历测试数据集，并使用模型进行预测。
    * 获取预测结果和真实标签，计算准确率。
    * 最后打印测试集上的准确率。

这段代码示例了如何使用 PyTorch 对 BERT 模型进行评估，它提供了计算准确率的函数，并使用 `torch.no_grad()` 上下文管理器来禁用梯度计算，提高评估效率。


----

In [ ]:
# 创建一个命名空间对象，用于存储训练模型需要的参数。
args = SimpleNamespace(num_labels=3, batch_size=16, test_batch_size=10, epochs=3, lr=2e-5, seed=1,log_interval =50, model_dir = "model/", data_dir="data/", num_gpus=1, train_file = "train.csv", test_file="test.csv")      

# 使用 train 函数训练模型。
model = train(args) 

**解释：**

这段代码设置了模型训练所需的各种参数，并调用 `train` 函数进行模型训练。

1. **设置训练参数：**  使用 SimpleNamespace 创建一个命名空间对象 `args`，并设置其属性，用于存储模型训练所需的各种参数：
    * **`num_labels`：**  模型需要预测的类别数量。
    * **`batch_size`：**  训练数据的批次大小，指定每次训练时加载到模型中的样本数量。
    * **`test_batch_size`：**  测试数据的批次大小，指定每次测试时加载到模型中的样本数量。
    * **`epochs`：**  训练的轮数。
    * **`lr`：**  学习率，控制模型参数更新的速度。
    * **`seed`：**  随机数种子，确保每次训练结果的一致性。 
    * **`log_interval`：**  日志打印间隔，指定每隔多少步打印一次训练信息。
    * **`model_dir`：**  存储训练好的模型的目录。
    * **`data_dir`：**  存储训练数据和测试数据的目录。
    * **`num_gpus`：**  使用的 GPU 数量。
    * **`train_file`：**  训练数据集的文件名。
    * **`test_file`：**  测试数据集的文件名。

2. **训练模型：** 调用 `train` 函数，并传入设置好的参数 `args`，开始模型的训练过程。

这段代码展示了如何使用命名空间对象组织模型训练参数，并调用 `train` 函数启动模型训练，为后续的自然语言处理和模型应用搭建了基础。


----

In [ ]:
def input_fn(request_body, request_content_type): 
    # 检查请求内容类型是否为 JSON 格式。
    if request_content_type == "application/json": 
        # 解析 JSON 请求体。
        data = json.loads(request_body)     

        # 检查解析后的数据类型是否为字符串或者列表。
        if isinstance(data, str): 
            # 如果数据类型为字符串，则将其封装成一个元素的列表。
            data = [data] 

        elif isinstance(data, list) and len(data) > 0 and isinstance(data[0], str): 
            # 如果数据类型为非空列表，且列表元素为字符串，则保持原样。
            pass 

        else: 
            # 抛出异常，因为数据类型不支持。
            raise ValueError("不支持的输入类型。输入类型必须为字符串或非空列表。我得到了 {}。".format(data)) 

        # 加载预训练的 BERT tokenizer。
        tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", do_lower_case=True) 

        # 将文本数据转换成 token id。
        input_ids = [tokenizer.encode(x, add_special_tokens=True) for x in data] 

        # 对较短的文本进行填充，使其长度与最长的文本长度一致。
        padded =  torch.zeros(len(input_ids), MAX_LEN)  
        for i, p in enumerate(input_ids): 
            padded[i, :len(p)] = torch.tensor(p) 

        # 创建注意力掩码，用于标识文本中的有效 token。
        mask = (padded != 0) 

        # 返回 token id 和注意力掩码，用于后续模型预测。
        return padded.long(), mask.long() 

    # 抛出异常，因为不支持当前的内容类型。
    raise ValueError("不支持的内容类型: {}".format(request_content_type))

**解释：**

这段代码定义了一个名为 `input_fn` 的函数，用于处理来自模型端点的请求，并将其转换成模型可以接受的输入数据。

* **函数参数：** 该函数接受两个参数：`request_body` 表示请求的正文，`request_content_type` 表示请求的内容类型。
* **检查内容类型：**  函数首先检查请求内容类型是否为 JSON 格式。
* **解析数据：**  如果内容类型为 JSON 格式，则解析请求正文，并将数据转换成字符串列表或单个字符串。
* **预处理数据：**  使用预训练的 BERT tokenizer 将文本转换成 token id，并对较短的文本进行填充，使其长度与最长的文本长度一致。最后创建注意力掩码，用于标识文本中的有效 token。
* **返回结果：**  函数返回处理后的 token id 和注意力掩码，用于后续的模型预测。

这段代码展示了如何处理模型端点的请求，并使用 BERT tokenizer 对文本进行预处理，以便将数据有效地使用在模型预测中。


----

In [ ]:
def predict_fn(input_data, model): 
    # 设置预测设备，如果 GPU 可用则使用 CUDA，否则使用 CPU。
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 

    # 将模型加载到指定设备上。
    model.to(device) 

    # 将模型设置为评估模式。
    model.eval() 

    # 从输入数据中获取 token id 和注意力掩码。
    input_id, input_mask = input_data 

    # 将 token id 和注意力掩码加载到指定设备上。
    input_id = input_id.to(device) 
    input_mask = input_mask.to(device) 

    # 禁用梯度计算，因为在预测过程中不需要计算梯度。
    with torch.no_grad(): 
        # 使用模型进行预测。
        y = model(input_id, attention_mask=input_mask)[0] 

    # 返回预测结果。
    return y 

**解释：**

这段代码定义了一个名为 `predict_fn` 的函数，用于使用训练好的 BERT 模型对输入数据进行预测，并返回预测结果。

* **函数参数：**  函数接收两个参数：`input_data` 表示预处理后的输入数据，`model` 表示训练好的模型。
* **模型预测：**  函数首先将模型加载到指定的设备上，然后将模型设置为评估模式。接着，函数接收预处理后的输入数据，包括 token id 和注意力掩码，并将它们加载到指定的设备上。然后，函数使用训练好的模型对输入数据进行预测，并返回预测结果。

这段代码展示了如何使用 PyTorch 和训练好的 BERT 模型进行预测。它将模型加载到合适的设备上，并禁用梯度计算，以提高预测效率。


----

In [ ]:
# 导入 json 库，用于处理 JSON 格式数据。
import json

# 定义一个测试文本。
article = "Operating profit outpaced the industry average" 

# 将测试文本转换为 JSON 格式字符串。
request_body = json.dumps(article) 

# 使用 input_fn 函数对文本进行预处理，并获取 token id 和注意力掩码。
enc_data, mask = input_fn(request_body, 'application/json') 

# 使用模型对预处理后的数据进行预测。
output = predict_fn((enc_data, mask), model) 

# 将预测结果转换为 numpy 数组。
preds = output.detach().cpu().numpy() 

# 打印预测结果，即情感标签。
print("情感标签: " + str(np.argmax(preds))) 

**解释：**

这段代码演示了如何使用训练好的模型对新的文本数据进行情感分类预测。

* **准备输入数据：** 首先定义一个测试文本 `article`，并使用 `json.dumps` 将其转换为 JSON 格式字符串，模拟模型部署后的请求格式。
* **数据预处理：**  使用 `input_fn` 函数对 JSON 格式的文本数据进行预处理，将其转换为模型可以接受的输入数据，包括 token id 和 attention mask。
* **模型预测：** 使用 `predict_fn` 函数加载训练好的模型，并对预处理后的数据进行预测，得到预测结果 `output`。
* **结果处理：** 将预测结果 `output` 转换为 NumPy 数组，并使用 `np.argmax` 获取预测结果中概率最大的类别索引，该索引即为模型预测的情感标签。最后打印预测的情感标签。

这段代码展示了如何使用训练好的 BERT 模型对新的文本数据进行情感分类预测，并展示了完整的预测流程，包括数据预处理、模型预测和结果处理。


----